In [ ]:
import functions as f
import pandas as pd
import geopandas as gpd
import rtree

In [ ]:
input_data_paths='/home/craig/footpath_width_analysis/data_input/mastermap2017_ncl_roadspaths_sample2.geojson'
paths_centerlines='dev_centrelines.shp'
paths_polygon_widths='dev_widths.shp'

input_road_network = '/home/craig/footpath_width_analysis/data_input/openroads_ncl_sample2.geojson'
buffered_roads='buffered_roads.shp'
roads_with_pavement_stats = 'road_polygons_pavement_stats.shp
road_network_with_pavement_stats ='result_openroads.shp'

In [ ]:
# generate centerlines and widths for pavement sections
df = gpd.read_file(input_data_paths)

df['centerlines'] = f.gen_centerlines(df)
print('Done centerline method')
df.centerlines = df.centerlines.apply(f.linemerge)
print('Done linemerge')
df.centerlines = df.centerlines.apply(f.ProcessingMethod.remove_short_lines)
print('Done remove short lines')
df.centerlines = df.centerlines.apply(lambda line: line.simplify(1, preserve_topology=True))
print('Done simplify')
df['segments'] = df['centerlines'].apply(f.get_segments)
print('Done get segments')
df['avg_distances'] = df.apply(f.get_avg_distances, axis=1)
print('Done get avg distances')
dfc = df.set_geometry('centerlines')
df_segments = f.ProcessingMethod.explode_to_segments(df)
dfc_segments = f.ProcessingMethod.explode_to_segments_(dfc)
print('Saving files')
df_segments.to_file(paths_polygon_widths)
dfc_segments.to_file(paths_centerlines)

In [ ]:
df_segments.sindex
dfc_segments.sindex
df.sindex

In [ ]:
# read in road data (centerlines for roads)
df2 = gpd.read_file(input_road_network)
#print(df2.head())
data = {'geometry':[], 'i':[], 'avg_width':[], 'min_width':[]}
for i, row in df2.iterrows():

    buffer_by = 0.0
    buffer_by = row.averagewid
    if buffer_by == 0: buffer_by = 5
    data['geometry'].append(row.geometry.buffer((buffer_by*0.5)+0.5))

    data['i'].append(i)
    data['avg_width'].append(row.averagewid)
    data['min_width'].append(row.minimumwid)
    

df2_ = pd.DataFrame(data)
df2_ = gpd.GeoDataFrame(df2_, crs=df.crs, geometry='geometry')

df2.sindex
df2_.to_file(buffered_roads)

In [ ]:
# slim down the dataframe to only required fields
data = {'geometry':[], 'i':[], 'width':[]}
for i, row in df_segments.iterrows():
        data['geometry'].append(row.geometry)
        data['i'].append(i)
        data['width'].append(row.width)

df__ = pd.DataFrame(data)
df__ = gpd.GeoDataFrame(df__, crs=df.crs, geometry='geometry')
    

In [ ]:
# get the overlapping pavement polygons (df__, i_2) for each road (df2, i_1)
df4 = gpd.overlay(df2_, df__, how='intersection')

In [ ]:
# using the result of the overlap, for each road section get the pavements around it
row_count = 0
for i, row in df2_.iterrows():
    #if row.i_1 == 0:
    #    print(row)
    row_count += 1
        
#print(row_count)
data = {'i_1':[], 'count':[], 'path_avg_width':[], 'geometry':[], 'total':[], 'intersects':[], 'path_max_width':[], 'path_min_width':[]}
for j in range(0, row_count):
    #print(j)
    data['geometry'].append(df2_.loc[j, 'geometry'])
    data['i_1'].append(j)
    count= 0
    pv_width=0.0
    min_width = 999999
    max_width = -999999
    intersects = []
    for i, row in df4.iterrows():
        if row.i_1 == j:
            count +=1
            pv_width += df__.loc[row.i_2, 'width']
            intersects.append(row.i_2)
            if df__.loc[row.i_2, 'width'] > max_width:
                max_width = df__.loc[row.i_2, 'width']
            if df__.loc[row.i_2, 'width'] < min_width:
                min_width = df__.loc[row.i_2, 'width']
    
    data['count'].append(count)
    data['intersects'].append(str(intersects))
    data['total'].append(pv_width)
    data['path_min_width'].append(min_width)
    data['path_max_width'].append(max_width)
    
    if count > 0:
        # to get average sum up intersecting pavements, these are on both sides so divide by 2, and by the number of pavements???
        # sooooo many issues here - it should be weighted average based on length along road segment
        # - for not all roads, will the total include pavements at both sides
        data['path_avg_width'].append((pv_width/count)/2)
    else: data['path_avg_width'].append(0)
        
df5 = pd.DataFrame(data)
df5 = gpd.GeoDataFrame(df5, crs=df.crs, geometry='geometry')
df5.to_file(roads_with_pavement_stats)

In [ ]:
df5.head()

In [ ]:
# generate df6 - a dataframe of the road centerlines with the pavement width/intersection data
df6=df5
data = {'geometry':[]}
for i, row in df6.iterrows():
    
    data['geometry'].append(df2.loc[row.i_1, 'geometry'])
    
df6.geometry=data['geometry']
#df6.head()
df6.to_file(road_network_with_pavement_stats)

In [ ]:
# problems to solve
# (a) pavement on different side of road
    # get centerline of road buffer
    # or offset road by
# (b) the average width is based on the values which intersect, and not the length of the intersection, so each pavement width is equally weighted